## <span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">Assignment 7: Bank Churn Prediction </span>


<p align="center" style="font-family: Arial;color:#0e92ea;font-size:1em;">
Given a Bank customer, build a neural network-based classifier that can determine whether they will leave or not in the next 6 months.
</p>

##  <span style="font-family: Arial; font-weight:bold;font-size:1.3em;color:#0e92ea">Data Dictionary:</span>

<ol style="font-family: Arial;color:#0e92ea;font-size:1em;">
    <li>CustomerId: Unique ID which is assigned to each customer</li>
    <li>Surname: Last name of the customer</li> 
    <li>CreditScore: It defines the credit history of the customer.</li>  
    <li>Geography: A customer’s location</li>    
    <li>Gender: It defines the Gender of the customer</li>   
    <li>Age: Age of the customer</li>     
    <li>Tenure: Number of years for which the customer has been with the bank</li>
    <li>NumOfProducts: It refers to the number of products that a customer has purchased through the bank.</li>
    <li>Balance: Account balance</li>
    <li>HasCrCard: It is a categorical variable that decides whether the customer has a credit card or not.</li>
    <li>EstimatedSalary: Estimated salary</li> 
    <li>isActiveMember: It is a categorical variable that decides whether the customer is an active member of the bank or not ( Active member in the sense, using bank products regularly, making transactions, etc )
    <li>Excited: It is a categorical variable that decides whether the customer left the bank within six months or not. It can take two values</li>
        <ul>
            <li>0=No ( Customer did not leave the bank )</li>
            <li>1=Yes ( Customer left the bank )</li>
        </ul>
    </li>
</ol>

In [1]:
import keras
import warnings
import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

warnings.filterwarnings('ignore')
main_data = pd.read_csv('Dataset - bank.csv')

In [2]:
main_data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [3]:
main_data.shape

(10000, 14)

In [10]:
'''
Input:
Pandas DataFrame

Output:
Displays DataFrame structure 
(columns, nulls and non nulls counts and percentage highlighing columns with most number of nulls)

Retunrs:
N/A
'''
def info(dataFrame):
    print(f"{dataFrame.shape[0]} Rows x {dataFrame.shape[1]} Columns")
    missing_values             = dataFrame.isna().sum()         # Get a series counting number of empty values for each column
    non_missing_values         = dataFrame.notnull().sum()      # Get a series counting number of non empty valuesfor each column 
    missing_values_percentage  = ((missing_values * 100)/(missing_values + non_missing_values)).astype(float)
    column_datatypes           = dataFrame.dtypes               # Get a series containing data types for each column 
    
    features      = [missing_values, missing_values_percentage, column_datatypes]
    feature_names = ["No. Of Missing Values", "Missing Values %", "Data Type"]
    
    df = pd.concat(
        objs = features,
        axis = 1,
        keys = feature_names, 
        sort = True)
    
    cm = sns.light_palette("red", as_cmap=True)
    display(df.style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, feature_names[1]]).format(formatter={(feature_names[1]): "{:.2f}%"})) 

In [11]:
info(main_data)

10000 Rows x 14 Columns


,No. Of Missing Values,Missing Values %,Data Type
Age,0,0.00%,int64
Balance,0,0.00%,float64
CreditScore,0,0.00%,int64
CustomerId,0,0.00%,int64
EstimatedSalary,0,0.00%,float64
Exited,0,0.00%,int64
Gender,0,0.00%,object
Geography,0,0.00%,object
HasCrCard,0,0.00%,int64
IsActiveMember,0,0.00%,int64
